# Linuxの操作メモ
忘れがちだけど時たま使う操作や苦手な操作をメモしておく。  

参考：  
* [1日1問、半年以内に習得　シェル・ワンライナー160本ノック](https://www.amazon.co.jp/gp/product/B09GTV35VJ/ref=ppx_yo_dt_b_d_asin_title_351_o00?ie=UTF8&psc=1)

# ■ 基本

## 任意のディレクトリ配下の容量を表示
duコマンドに-hオプションをつけることでSI単位で容量を表示する。

```
 du -h 容量を調べたいディレクトリ
```

## 複数ディレクトリ配下の容量をサマライズして表示
下記コマンドを実行するとカレントディレクトリにある全ディレクトリ配下の容量（SI単位）を昇順で表示してくれる。  

```
 du -sh */ | sort -h
```

## 任意のディレクトリ配下のうち、大きいサイズのディレクトリを調べる
sort -rnは-n:numericに比較、-r: 降順に並べるの意味。

```
du . | sort -rn | head -10
```

## 連番で空ファイルを作成する
```
for i in $(seq 3); do touch file_$i.txt ;done
```

## ディレクトリのみ表示する
```
ls -d */
```

In [4]:
! ls

AA101  AA102  AA201  file_1.txt  file_2.txt  file_3.txt  practice_memo.ipynb


In [1]:
! ls -d */

AA101/	AA102/	AA201/


## カレントディレクトリの全ディレクトリに空ファイルを作成する

In [5]:
! cat touch_file.sh

#!/bin/sh

dir_list=$(ls -d */ | cut -c 1-5)

for dir in $dir_list;
do
	for i in $(seq 3);
	do
		touch ./$dir/file_$i.txt;
	done;
done;



## ファイルを1行ずつ処理する

1行ずつファイルの中身をechoで標準出力する例。
```
 cat ファイル名 | while read line; do echo $line; done
```

## ファイルの1行目に任意の文字列を入れる

下記のように書けば良い。  
1iで1行目に追記するの意。2iとすれば２行目に挿入される。    

```
sed -i '1i 入れたい文字列' 対象ファイル名
```

## xargsで複数ファイルをまとめて操作

### ・例1：カレントディレクトリにある拡張子が.txtのファイルをtempディレクトリへコピー  

```
ls *.txt | xargs -I{} cp {} temp
```
-I{}で{}の位置にパイプからの引数を入れるという操作になる。

### ・例2： findの結果にヒットしたファイルを削除

```
find temp -regex ".*file_[12].*" | xargs -I{} rm {}
```
上記ではtemp配下のfile_1.txtまたはfile_2.txtを削除している。

### ・例3： 削除したいファイルのリストを読み込んで削除
```
cat rm_files_list.txt | xargs -I{} rm {}
```

### ※補足
-pオプションをつけることで、ドライランで実際に実行されるコマンドを確認できる

```
find temp -regex ".*file_[12].*" | xargs -I{} -p rm {}
```

## 2つのディレクトリを同期する
rsyncを使うことで、ディレクトリ内の各ファイルのタイムスタンプ、サイズを比較して  
変更があった分だけ同期できる。

```
rsync -av 同期元/ 同期先/
```
-aはサブディレクトリも含めて再帰的に処理するオプション。  
パーミッションやファイルのタイムスタンプも保持される。  
-vはverbose。同期元と同期先の最後に/をつけないと、  
同期元のフォルダ自体と等しいか否かを判定してしまい、まるごと同期されてしまう。

## 既にファイルがあったら処理をしない
例として、ディレクトリのリストがあり、これらのディレクトリをtar -cvzfでアーカイブ化＆圧縮したいが、  
既に処理済みのものは処理をスキップしたい場合を考える。  
以下のようにif文でtarファイルの存在有無を確認すればよい。

In [9]:
! cat practice-2_Hanling_list_of_files/execute_tar.sh

#!/bin/bash

target_dirs=$(ls -d */)

for dir in $target_dirs;
do
	base_dir=$(basename $dir) 
	if [ -e ${base_dir}.tar ]; then
		echo "${base_dir}.tar already exists. skip tar."
	else
		tar -czvf ${base_dir}.tar $dir
	fi
done


## renameコマンドによるファイルのリネーム
mvコマンドではなくrenameコマンドを使うとsedっぽくファイル名を変更できる。  
renameコマンドはデフォルトでインストールされていない場合があるので注意。  

例）abc.txtのa or bが２回続く部分をxにrenameする。
```
rename 's/[ab]{2}/x/' abc.txt
```

# ■ システム系

## プロセスを特定の論理CPU上で動かす
下記を実行すると、<コマンド>引数で指定したコマンドを-c <論理CPU番号>引数で指定した  
CPU上で実行できる。特定のコアにバインドしてプログラムを動かしたいときに使える。
```
taskset -c <論理CPU番号> <コマンド>
```

## コマンドをバックグランドで実行する
&を末尾につけて実行すればよい。  
任意のコマンドを実行しつつターミナルで作業したいときに使える。  
```
<コマンド> &
```

## プロセスが発行するシステムコールを確認
プロセスがどんなシステムコールを発行するかはstraceコマンドで確認できる。
```
strace -o 出力先 プログラムのパス
```
straceの出力は１行１行が１システムコールに対応している。  
また、-Tオプションをつけることで各システムコールにかかった時間、  
-ttでシステムコールの発行時刻をマイクロ秒単位で確認可能。

## システムに存在する全プロセスを列挙する
```
ps aux
```
※auxの意味：
* a: 端末操作のプロセス(自分 + 他ユーザー)を表示する
* u: 各プロセスの実行ユーザーやCPU, Mem(メモリ)等の情報も表示する
* x: 端末操作のないプロセス(daemon等)も表示する

さらに f をつけて実行するとプロセス間の親子関係をtreeの形で表示できる。

------------------------------------------------------------

# ■ テキスト処理テクニック系

## 改行コードの確認
```
file ファイル名

# 上記で出力されなかった場合はodコマンドを使う
od -c ファイル名

0000000 343 201 231 343 201 232   :   :  \n 343 201 202 343 201 260 343
0000020 201 260 343 201 202 343 201 260 343 201 260   :   :  \n   :   :

→ 改行コードが\nだと分かる。
```

※odコマンドはファイルを8進数や16進数で表示するコマンド。  
https://developers-book.com/2020/11/22/407/

## 項目ごとの件数を数える
pandaやpolarsのvalue_counts()的な操作はawk, sort ,uniqを使えばよい。  
・例1）1～5までの数字の偶数と奇数を数える。
```
seq 5 | awk '{print $1%2 ? "odd":"even"}' | sort | uniq -c

# 結果
2 even
3 odd
```
※ 上記はawkの三項演算子（条件 ? "真の場合":"偽の場合"）を用いている。

uniqは重複行が連続していることを前提としているため、  
uniqの前にsortでodd, evenを辞書順に並び変える必要があることに注意。  
uniq -cは重複行を数えるオプション。  

・例2）1～5までの数字の偶数と奇数を数えて、降順に並べる
```
seq 5 | awk '{print $1%2 ? "odd":"even"}' | sort | uniq -c | awk '{print $2 , $1}' | sort -k2,2 -nr

# 結果
odd 3
even 2
```
-k2,2は2列目から2列目を基準にソートするという意味であり、ここでは数え上げ結果を基準にして、  
-nで数字順に比較し、-rで降順に並べ手出力している。（デフォルトでは昇順）  

※補足　awkだけで数える場合
```
seq 5 | awk '{print $1%2 ? "odd":"even"}' | awk '{a[$1]++} END{for (k in a) print k, a[k]}'
```  
上記ではoddかevenかをキーとした連想配列を作って、$1に出てきた回数分valueをカウントしている。



## パディング
・例1）連番のファイル名を3桁にパディングしてリネーム（大量ファイル想定）
```
find -type f | xargs -i basename {} | awk '{printf("%d %03d\n",$1,$1)}' | xargs -n2 mv
```

## awkのfor文
・例1）階段上に文字を出力する例
```
$ seq 10 | awk '{for(i=10; $1<i; i--) {printf " "};  print "x"}'

出力：
         x
        x
       x
      x
     x
    x
   x
  x
 x
x
```  

printは改行あり、printfは改行なしであることに注意。

## 時刻のフィルタリング
### 例１）  
”[dd/m（英）/yyyy HH:MM:SS]”のような形式でタイムスタンプが記録されたログファイルがあり、  
2016年12月(Dec)24日21時から2016年12月(Dec)25日3時までの時間帯だけ抽出したい場合。

### 解法1: grepで頑張る
```
grep -E " \[24/Dec/2016 2[1-3].*\]|\[25/Dec/2016 0[0-3].*\] " log_range.log
```  
時刻範囲が広くなければこれでもOKだが、広いとOR条件が増えるので面倒になる。  
ただ、何らかの要因で時刻順にログが記録されていなくても、対象時間帯に該当するレコードだけ抜き出せるというメリットはある。  


### 解法2: sed -n '/正規表現1/,/正規表現2/p'を使う
```
cat log_range.log | sed -n '/24\/Dec\/2016 21.*/,/25\/Dec\/2016 03.*/p'
```

sed -n '/正規表現1/,/正規表現2/p'は正規表現1にマッチする行から正規表現2にマッチする行まで抜き出す操作になる。  
マッチした2点の間を抜き出しているだけなので、ログが時刻順になっていない、かつ、関係ない時刻が途中に含まれている場合は  
時刻をソートしてから実行する必要あり。  

### 解法3: awk '/正規表現1/,/正規表現2/p'を使う
```
awk '/24\/Dec\/2016 21.*/,/25\/Dec\/2016 03.*/' log_range.log  
```
解法2のような記法はawkのパターン部分でも出来る。  


## 正規表現にマッチした次の行に何か足す
例）Atx形式のマークダウンの見出し(#等)をSetext形式(見出しの前に===など)に変換する例。
```
sed -E 's/^#{1} (.*)/\1\n====/g' headings.md | sed -E 's/^#{2} (.*)/\1\n---/g'
```  
\1で後方参照を行い、\nをつけて次の行に書き足す内容を記述している。  

## bashの連想配列（辞書型）取り扱い

```
# 宣言
declare -A names=(["すず"]="鈴木" ["さと"]="佐藤" ["やま"]="山田")

# value取り出し
echo ${names["すず"]}

鈴木
```  


## 改行コードをいじる

・例１）改行コードLFを”：”に変更したい場合
```
sed -E -z 's/\n/:/g' input.txt
```
sedは改行コード単位で分割された行の内容を置換するため、デフォルトでは改行コード自体は置換対象に含まれない。  
含めるには-zオプションを使用する。 

・例２）改行コードCRLFをLFに変更したい場合  
Windows環境で作成したテキストファイルをLinux環境で問題なく使う場合などを想定。  
```
sed -z 's/\r\n/\n/g' input.txt
```

参考: https://webbibouroku.com/Blog/Article/sed-crlf-lf

## 数行ごとに分かれた内容を1行にまとめたい
例えば以下のようなファイルがあったとき、
key1 value1  
key2 value2  
のように表示したいときを考える。
```
$ cat example.txt
key1
value1

key2
value2

key3
value3
```  

これは下記のようにxargsで読み込む引数に2を指定すればよい。  
xargsは何もコマンドを指定しないとechoの動作になるため、このようになる。  
3行の内容を1行にするのも-n3とすればよい。  

```
cat example.txt | xargs -n2

# 出力結果
key1 value1
key2 value2
key3 value3
```



## sedでまとめて置換する
下記のように一つシングルクォーテーションの中をセミコロンで区切ればできる。

```
sed  's/^すず/鈴木/g; s/^さと/佐藤/g; s/^やま/山田/g'
```  
